In [1]:
import os
import datetime as dt
import time
import json

import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn import metrics

# https://stackoverflow.com/questions/14254203/mixing-categorial-and-continuous-data-in-naive-bayes-classifier-using-scikit-lea?rq=1
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB
from sklearn.ensemble import ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier

import xgboost as xgb
from xgboost import XGBClassifier
import lightgbm as lgb
from lightgbm import LGBMClassifier

In [2]:
today = dt.date.today()

In [3]:
# should've done this in cleaning
# rscaler = RobustScaler()
# rscaler.fit(x_train)

# x_train_scaled = rscaler.transform(x_train)
# x_test_scaled = rscaler.transform(x_train)

## Overview

This notebook is going to be used a few times to get parameters for models and test out different configurations or sets of features. 

The first time through I tried out a bunch of different algorthims (mainly tree algorithms) and parameters.
I attempted to grid-search parameters, but doing so was to resource intensive and would take days to finish even for only a modest parameter search space. I ended up running bayesian optimizations on the model parameters and found that LightGBM was the best combination of fast and high auc (approximately the same as xgboost and a fraction of the time).

For further explorations and iterations I stuck with LGBM.

### Config Vars

In [28]:
PK = 'sk_id_curr'
TARGET = 'target'
N_CV = 4
SEED = 1111
DATA_DIR = 'clean_data/'

TRAIN_FILE = os.path.join(DATA_DIR, 'mrgd_train.csv')
TEST_FILE = os.path.join(DATA_DIR, 'mrgd_test.csv')
SUBMISSION_OUTPUT_FILE = os.path.join(DATA_DIR, 'submission_out.csv')

DTYPES = {'sk_id_curr':str, 'sk_id_bureau':str, 'sk_id_prev':str,'num_instalment_version':str}



### Load Data

In [5]:
train = pd.read_csv(TRAIN_FILE, dtype=DTYPES)
test = pd.read_csv(TEST_FILE, dtype=DTYPES)
train.shape, test.shape

((307511, 527), (48744, 526))

In [6]:
train.head()

,sk_id_curr,flag_own_car,flag_own_realty,name_contract_type,flag_cont_mobile,flag_document_10,flag_document_11,flag_document_12,flag_document_13,flag_document_14,...,sk_dpd_pos,name_contract_status_Active,name_contract_status_Amortized debt,name_contract_status_Approved_pos,name_contract_status_Canceled_pos,name_contract_status_Completed_pos,name_contract_status_Demand_pos,name_contract_status_Returned to the store,name_contract_status_Signed_pos,target
0,100002,0,1,0,1,0,0,0,0,0,...,0.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1
1,100003,0,0,0,1,0,0,0,0,0,...,0.0,26.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0
2,100004,1,1,1,1,0,0,0,0,0,...,0.0,3.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0
3,100006,0,1,0,1,0,0,0,0,0,...,0.0,18.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,0
4,100007,0,1,0,1,0,0,0,0,0,...,0.0,62.0,0.0,0.0,0.0,3.0,0.0,0.0,1.0,0


Separate keys, target, train and test

In [8]:
# Target var
y_train = train[TARGET].values
pd.value_counts(y_train)

0    282686
1     24825
dtype: int64

In [9]:
# sk ids
train_id = train[PK]
test_id = test[PK]

In [10]:
train.drop([PK, TARGET], axis=1, inplace=True)
test.drop(PK, axis=1, inplace=True)

In [11]:
print("{},{}".format(train.shape, test.shape))

(307511, 525),(48744, 525)


In [13]:
features = train.columns

In [14]:
# make arrays
x_train = train[features].values
x_test = test[features].values
###
x_train = train.values
x_test = test.values

### Grid Search Params

In [15]:
from sklearn.model_selection import GridSearchCV

In [16]:
et_default_params = {
    'n_jobs': -1,
    'verbose' : 1
}

rf_default_params = {
    'n_jobs': -1,
    'verbose' : 1
}

gb_default_params = {
    'verbose' : 1
}

xgb_default_params = {
    'silent': 0,
    'n_jobs': -1,
    'objective': 'binary:logistic',
    'eval_metric': 'auc'
}

lgb_default_params = {
    'silent': 0,
    'n_jobs': -1,
    'objective': 'binary',
    'metric': 'auc',
    #'is_unbalanced':True
}

In [17]:
et_params = {
    'n_estimators': list(range(25,201, 25)),
    'max_features': [0.01, 0.05, 0.1, 0.3, 0.5, 0.75],
    'max_depth': list(range(5,26, 5)),
}

rf_params = {
    'n_estimators': list(range(25,201, 25)),
    'max_features': [0.01, 0.1, 0.5, 0.8],
    'max_depth': list(range(5,26, 5)),
}


gb_params = {
    'n_estimators': range(20, 101, 20),
    'learning_rate': [0.01, 0.1, 0.5, 0.8],
    'max_features': [0.01, 0.1, 0.5, 0.8],
    'max_depth': list(range(5,26, 5)),
}


xgb_params = {
    'colsample_bytree': [0.01, 0.1, 0.5, 0.8],
    'subsample': [0.01, 0.1, 0.5, 0.8],
    'learning_rate': [0.01, 0.1, 0.5, 0.8],
    'max_depth': list(range(5,26, 5)),
    'scale_pos_weight': range(2, int(pd.value_counts(y_train)[0]/pd.value_counts(y_train)[1])*2, 4),
    'nrounds': [2**n for n in range(5,9)]
}

lgb_params = {
    'boosting': ['gbdt', 'dart'],
    'num_iterations': [2**n for n in range(5,10)],
    'learning_rate': [0.01, 0.1, 0.5, 0.8],
    'max_depth': list(range(5,26, 5)),
    'scale_pos_weight': range(2, int(pd.value_counts(y_train)[0]/pd.value_counts(y_train)[1])*2, 4),
    'subsample': [0.01, 0.1, 0.5, 0.8],
    'colsample_bytree': [0.01, 0.1, 0.5, 0.8],
}

# potential other params afterwords
# 'lambda_l1': [0, 0.6,]
# 'lambda_l2': [0, 0.6,]

In [18]:
np.prod(list(map(len, xgb_params.values())))

6400

In [19]:
# OVERSAMPLE
def get_weights(ser):
    vcs = ser.value_counts()
    class_labels = vcs.index.tolist()
    class_sizes = vcs.values
    nclasses = len(class_labels)
    class_weights = dict(zip(class_labels,(nclasses**-1/class_sizes)))
    return class_weights

def oversample(ser, size):
    p = ser.map(get_weights(ser))
    samp = np.random.choice(ser.index,size,p = p)
    return samp


def grid_search_params(algo, default_params, grid_params, cv=N_CV, 
                       x_train=x_train, y_train=y_train):
    est = algo(**default_params)
    grid = GridSearchCV(est, grid_params, scoring='roc_auc', cv=cv, n_jobs=-1, )
    print('Grid Searching Params')
    
    start = time.time()
    grid.fit(x_train, y_train)
    print('Fitting Completed in {} minutes'.format((time.time()-start)/60))
    print('Best Score', grid.best_score_)
    
    return grid.best_estimator, grid.best_params

In [18]:
# results={}

In [19]:
# names = ['et', 'rf', 'gb', 'xgb', 'lgb']
# algos = [ExtraTreesClassifier, RandomForestClassifier, GradientBoostingClassifier, XGBClassifier, LGBMClassifier]
# default_params = [et_default_params, rf_default_params, gb_default_params, xgb_default_params, lgb_default_params]
# param_grids = [et_params, rf_params, gb_params, xgb_params, lgb_params]


# for name, algo, dparams, gparams in zip(algos,default_params,param_grids):
#     best_estimator, best_params = grid_search_params(algo, dparams, gparams)
#     results[name] = (best_estimator, best_params)

### Bayesian Param Search

New to this - exploring using bayes_opt python package for finding optimal parameters

In [20]:
from bayes_opt import BayesianOptimization

In [21]:
all_results = {}

#### Random Forest

In [66]:
def rf_func(**params):
    params['n_estimators'] = int(np.round(params['n_estimators']))
    params['max_depth'] = int(np.round(params['max_depth']))
    
    
    est = RandomForestClassifier(**rf_default_params, **params)
    cv = StratifiedKFold(N_CV, True, SEED)
    roc_auc = np.mean(cross_val_score(est, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1))
    return roc_auc

In [23]:
{p: (min(rf_params[p]), max(rf_params[p])) for p in rf_params}

{'max_depth': (5, 25), 'max_features': (0.01, 0.8), 'n_estimators': (25, 200)}

In [24]:
start = time.time()
rfBO = BayesianOptimization(rf_func, {p: (min(rf_params[p]), max(rf_params[p])) for p in rf_params})
rfBO.maximize(10, 100)
print('TIME TAKEN (MIN):', (time.time() - start)/60)

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |   max_depth |   max_features |   n_estimators | 


[Parallel(n_jobs=-1)]: Done  22 out of  42 | elapsed:    7.9s remaining:    7.2s
[Parallel(n_jobs=-1)]: Done  22 out of  42 | elapsed:   11.3s remaining:   10.3s
[Parallel(n_jobs=-1)]: Done  22 out of  42 | elapsed:   11.0s remaining:   10.0s
[Parallel(n_jobs=-1)]: Done  42 out of  42 | elapsed:   16.4s finished
[Parallel(n_jobs=32)]: Done  22 out of  42 | elapsed:    0.1s remaining:    0.1s
[Parallel(n_jobs=32)]: Done  42 out of  42 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done  42 out of  42 | elapsed:   15.8s finished
[Parallel(n_jobs=32)]: Done  22 out of  42 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  42 out of  42 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done  42 out of  42 | elapsed:   13.7s finished
[Parallel(n_jobs=32)]: Done  22 out of  42 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  42 out of  42 | elapsed:    0.0s finished


    1 | 00m24s |    0.73414 |      7.4374 |         0.2544 |        41.5493 | 


[Parallel(n_jobs=-1)]: Done 184 out of 184 | elapsed:  1.0min finished
[Parallel(n_jobs=32)]: Done 184 out of 184 | elapsed:    0.7s finished
[Parallel(n_jobs=-1)]: Done 184 out of 184 | elapsed:  1.0min finished
[Parallel(n_jobs=32)]: Done 184 out of 184 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done 184 out of 184 | elapsed:  1.0min finished
[Parallel(n_jobs=32)]: Done 184 out of 184 | elapsed:    0.1s finished


    2 | 01m11s |    0.73283 |      5.8668 |         0.2426 |       183.9660 | 


...

  108 | 00m36s |    0.72244 |     10.9645 |         0.0209 |       119.3315 | 


[Parallel(n_jobs=-1)]: Done 189 out of 189 | elapsed: 10.9min finished
[Parallel(n_jobs=32)]: Done 189 out of 189 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Done 189 out of 189 | elapsed: 10.9min finished
[Parallel(n_jobs=32)]: Done 189 out of 189 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done 189 out of 189 | elapsed: 10.8min finished
[Parallel(n_jobs=32)]: Done 189 out of 189 | elapsed:    0.3s finished


  109 | 11m23s |    0.73110 |     22.6522 |         0.7916 |       188.5414 | 


[Parallel(n_jobs=-1)]: Done  77 out of  77 | elapsed:    3.6s finished
[Parallel(n_jobs=32)]: Done  77 out of  77 | elapsed:    0.4s finished
[Parallel(n_jobs=-1)]: Done  77 out of  77 | elapsed:    5.3s finished
[Parallel(n_jobs=32)]: Done  77 out of  77 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done  77 out of  77 | elapsed:    4.7s finished
[Parallel(n_jobs=32)]: Done  77 out of  77 | elapsed:    0.1s finished


  110 | 00m37s |    0.72954 |      8.5592 |         0.0424 |        77.4776 | 
TIME TAKEN (MIN): 378.82150808970135


In [25]:
result = rfBO.res['max']
# best_params = result['max_params']
# best_score = result['max_value']
print(result)
all_results['rf'] = result

#### Extra Trees

In [26]:
def et_func(**params):
    params['n_estimators'] = int(np.round(params['n_estimators']))
    params['max_depth'] = int(np.round(params['max_depth']))
    
    cv = StratifiedKFold(N_CV, True, SEED)
    est = RandomForestClassifier(**et_default_params, **params)
    roc_auc = np.mean(cross_val_score(est, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1))
    return roc_auc

In [27]:
start = time.time()
etBO = BayesianOptimization(et_func, {p: (min(et_params[p]), max(et_params[p])) for p in et_params})
etBO.maximize(10, 100)
print('TIME TAKEN (MIN):', (time.time() - start)/60)

Initialization
-----------------------------------------------------------------------------
 Step |   Time |      Value |   max_depth |   max_features |   n_estimators | 


[Parallel(n_jobs=-1)]: Done 171 out of 171 | elapsed:  2.7min finished
[Parallel(n_jobs=32)]: Done 171 out of 171 | elapsed:    0.5s finished
[Parallel(n_jobs=-1)]: Done 171 out of 171 | elapsed:  2.7min finished
[Parallel(n_jobs=32)]: Done 171 out of 171 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done 171 out of 171 | elapsed:  2.7min finished
[Parallel(n_jobs=32)]: Done 171 out of 171 | elapsed:    0.1s finished


    1 | 02m52s |    0.73736 |     10.2128 |         0.4233 |       171.3298 | 


[Parallel(n_jobs=-1)]: Done  30 out of  46 | elapsed:   10.1s remaining:    5.4s
[Parallel(n_jobs=-1)]: Done  30 out of  46 | elapsed:   13.0s remaining:    6.9s
[Parallel(n_jobs=-1)]: Done  30 out of  46 | elapsed:   12.3s remaining:    6.6s
[Parallel(n_jobs=-1)]: Done  46 out of  46 | elapsed:   17.8s finished
[Parallel(n_jobs=32)]: Done  30 out of  46 | elapsed:    0.2s remaining:    0.1s
[Parallel(n_jobs=32)]: Done  46 out of  46 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done  46 out of  46 | elapsed:   17.8s finished
[Parallel(n_jobs=32)]: Done  30 out of  46 | elapsed:    0.1s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  46 out of  46 | elapsed:    0.1s finished
[Parallel(n_jobs=-1)]: Done  46 out of  46 | elapsed:   15.8s finished
[Parallel(n_jobs=32)]: Done  30 out of  46 | elapsed:    0.0s remaining:    0.0s
[Parallel(n_jobs=32)]: Done  46 out of  46 | elapsed:    0.0s finished


    2 | 00m26s |    0.73989 |      9.5069 |         0.1887 |        45.9852 | 


...

  108 | 04m03s |    0.73197 |      9.4554 |         0.7500 |       141.4496 | 


[Parallel(n_jobs=-1)]: Done 134 out of 134 | elapsed:  5.3min finished
[Parallel(n_jobs=32)]: Done 134 out of 134 | elapsed:    0.3s finished
[Parallel(n_jobs=-1)]: Done 134 out of 134 | elapsed:  5.3min finished
[Parallel(n_jobs=32)]: Done 134 out of 134 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done 134 out of 134 | elapsed:  5.3min finished
[Parallel(n_jobs=32)]: Done 134 out of 134 | elapsed:    0.1s finished


  109 | 05m47s |    0.73597 |     14.9137 |         0.7500 |       134.0405 | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ 0.00020708]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)
[Parallel(n_jobs=-1)]: Done 113 out of 113 | elapsed:  6.1min finished
[Parallel(n_jobs=32)]: Done 113 out of 113 | elapsed:    0.6s finished
[Parallel(n_jobs=-1)]: Done 113 out of 113 | elapsed:  6.1min finished
[Parallel(n_jobs=32)]: Done 113 out of 113 | elapsed:    0.2s finished
[Parallel(n_jobs=-1)]: Done 113 out of 113 | elapsed:  6.1min finished
[Parallel(n_jobs=32)]: Done 113 out of 113 | elapsed:    0.2s finished


  110 | 06m37s |    0.72946 |     23.0671 |         0.7490 |       113.2034 | 
TIME TAKEN (MIN): 338.40679365793864


In [28]:
result = etBO.res['max']
# best_params = result['max_params']
# best_score = result['max_value']
print(result)
all_results['et'] = result

#### LGBM

In [29]:
lgbm_bounds = {p: (min(lgb_params[p]), max(lgb_params[p])) for p in lgb_params}
lgbm_bounds.update({'boosting': (0,1)})

In [36]:
def lgb_func(**params):
    params['boosting'] = 'gbdt' if np.round(params['boosting']) == 0 else 'dart'
    params['num_iterations'] = int(np.round(params['num_iterations']))
    params['max_depth'] = int(np.round(params['max_depth']))
    params['learning_rate'] = np.clip(params['learning_rate'], 0.0001, 1)
    params['subsample'] = np.clip(params['subsample'], 0.0001, 1)
    params['colsample_bytree'] = np.clip(params['colsample_bytree'], 0.0001, 1)    
    
    cv = StratifiedKFold(N_CV, True, SEED)
    est = LGBMClassifier(**lgb_default_params, **params)
    roc_auc = np.mean(cross_val_score(est, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1))
    return roc_auc

In [37]:
start = time.time()
lgbBO = BayesianOptimization(lgb_func, lgbm_bounds)
lgbBO.maximize(10, 100)
print('TIME TAKEN (MIN):', (time.time() - start)/60)

Initialization
-------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   boosting |   colsample_bytree |   learning_rate |   max_depth |    n_iter |   subsample | 
    1 | 00m18s |    0.76232 |     0.5101 |             0.1302 |          0.1547 |     11.3904 |  167.8788 |      0.0862 | 
    2 | 00m15s |    0.73637 |     0.4421 |             0.3621 |          0.5778 |      5.6535 |  208.4063 |      0.2464 | 
    3 | 00m17s |    0.74266 |     0.9955 |             0.1916 |          0.7767 |     21.1522 |  101.4697 |      0.6110 | 
    4 | 00m16s |    0.74563 |     0.3162 |             0.5576 |          0.3964 |     20.2816 |   36.8746 |      0.7254 | 
    5 | 00m15s |    0.75197 |     0.4680 |             0.1058 |          0.3785 |     18.3717 |  204.9472 |      0.1474 | 
    6 | 00m16s |    0.73944 |     0.1426 |             0.5686 |          0.4594 |     18.1782 |   45.2143 |      0.7810 | 
  

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -7.75064359e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


   22 | 00m31s |    0.75409 |     0.9965 |             0.7816 |          0.7368 |      5.1259 |  100.5630 |      0.4423 | 
   23 | 00m31s |    0.74520 |     1.0000 |             0.8000 |          0.8000 |     17.6830 |   32.0011 |      0.0100 | 
   24 | 00m26s |    0.68306 |     0.0000 |             0.0100 |          0.0100 |     25.0000 |  242.2520 |      0.0100 | 
   25 | 00m31s |    0.75209 |     1.0000 |             0.8000 |          0.8000 |      5.0000 |  228.1773 |      0.8000 | 
   26 | 00m28s |    0.72384 |     1.0000 |             0.0100 |          0.8000 |     16.1876 |  219.6935 |      0.8000 | 
   27 | 00m31s |    0.72825 |     0.0000 |             0.8000 |          0.8000 |     25.0000 |  189.9705 |      0.0100 | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([  3.33286953e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 5, 'warnflag': 2}
  " state: %s" % convergence_dict)


   28 | 00m35s |    0.73052 |     1.0000 |             0.8000 |          0.0100 |     25.0000 |   85.4162 |      0.8000 | 
   29 | 00m33s |    0.73670 |     0.0000 |             0.8000 |          0.0100 |     25.0000 |  208.7703 |      0.0100 | 
   30 | 00m34s |    0.73670 |     0.0000 |             0.8000 |          0.0100 |     10.7434 |  188.5835 |      0.0100 | 
   31 | 00m28s |    0.72381 |     1.0000 |             0.0100 |          0.8000 |     25.0000 |  124.0899 |      0.0100 | 


...

   82 | 00m35s |    0.74834 |     0.9786 |             0.7702 |          0.0754 |      5.2153 |  185.9534 |      0.2382 | 
   83 | 00m30s |    0.68306 |     0.0000 |             0.0100 |          0.0100 |     24.6706 |  201.7397 |      0.0886 | 
   84 | 00m38s |    0.75790 |     1.0000 |             0.8000 |          0.1040 |     12.7394 |  200.5241 |      0.2878 | 
   85 | 00m35s |    0.74822 |     1.0000 |             0.8000 |          0.8000 |      5.9681 |  216.0567 |      0.6989 | 
   86 | 00m37s |    0.76291 |     0.9664 |             0.7793 |          0.3565 |     15.6711 |  209.4841 |      0.0648 | 
   87 | 00m36s |    0.74869 |     0.9744 |             0.6897 |          0.6842 |     18.1926 |  194.3902 |      0.0615 | 
   88 | 00m36s |    0.74691 |     1.0000 |             0.7615 |          0.8000 |     20.6182 |  207.1825 |      0.8000 | 
   89 | 00m38s |    0.73670 |     0.0941 |             0.8000 |          0.0100 |     25.0000 |   98.9035 |      0.5715 | 
   90 | 00m39s |

/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -3.47263413e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 51, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


  102 | 00m37s |    0.73295 |     0.0000 |             0.8000 |          0.0100 |      5.0000 |   37.4880 |      0.8000 | 
  103 | 00m34s |    0.69739 |     0.3385 |             0.0338 |          0.0113 |     22.3734 |   78.6413 |      0.0714 | 
  104 | 00m36s |    0.69184 |     1.0000 |             0.0100 |          0.0100 |     25.0000 |  144.0939 |      0.0100 | 
  105 | 00m38s |    0.72966 |     0.0000 |             0.8000 |          0.8000 |     14.4045 |  132.0567 |      0.8000 | 
  106 | 00m43s |    0.73052 |     0.7483 |             0.8000 |          0.0100 |     25.0000 |   66.4621 |      0.8000 | 


/home/ec2-user/anaconda3/lib/python3.6/site-packages/sklearn/gaussian_process/gpr.py:427: UserWarning: fmin_l_bfgs_b terminated abnormally with the  state: {'grad': array([ -9.37493292e-05]), 'task': b'ABNORMAL_TERMINATION_IN_LNSRCH', 'funcalls': 55, 'nit': 4, 'warnflag': 2}
  " state: %s" % convergence_dict)


  107 | 00m40s |    0.75800 |     0.0046 |             0.2892 |          0.0308 |     24.9933 |  154.4825 |      0.6493 | 
  108 | 00m37s |    0.71657 |     0.0000 |             0.0100 |          0.8000 |     24.0799 |  174.5671 |      0.0100 | 
  109 | 00m39s |    0.73295 |     0.0000 |             0.8000 |          0.0100 |      5.0000 |  249.4574 |      0.0100 | 
  110 | 00m38s |    0.72744 |     0.0000 |             0.8000 |          0.8000 |     16.7493 |  104.8082 |      0.0100 | 
TIME TAKEN (MIN): 57.09322918256124


In [38]:
result = lgbBO.res['max']
# best_params = result['max_params']
# best_score = result['max_value']
print(result)
all_results['lgb'] = result

{'max_params': {'boosting': 1.0,
  'colsample_bytree': 0.21097363734776334,
  'learning_rate': 0.18490698058684293,
  'max_depth': 25.0,
  'n_iter': 157.84122032083656,
  'subsample': 0.010000000000000009},
 'max_val': 0.76478829553316874}

#### XGB

In [53]:
def xgb_func(**params):
    params['nrounds'] = int(np.round(params['nrounds']))
    params['max_depth'] = int(np.round(params['max_depth']))
    params['learning_rate'] = np.clip(params['learning_rate'], 0.0001, 1)
    params['subsample'] = np.clip(params['subsample'], 0.0001, 1)
    params['colsample_bytree'] = np.clip(params['colsample_bytree'], 0.0001, 1)    
    
    cv = StratifiedKFold(N_CV, True, SEED)
    est = XGBClassifier(**xgb_default_params, **params)
    roc_auc = np.mean(cross_val_score(est, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1))
    return roc_auc

In [ ]:
start = time.time()
xgbBO = BayesianOptimization(xgb_func, {p: (min(xgb_params[p]), max(xgb_params[p])) for p in xgb_params})
xgbBO.maximize(10, 100)
print('TIME TAKEN (MIN):', (time.time() - start)/60)

Initialization
------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |   learning_rate |   max_depth |   nrounds |   subsample | 


In [ ]:
result = xgbBO.res['max']
# best_params = result['max_params']
# best_score = result['max_value']
print(result)
all_results['xgb'] = result

In [34]:
with open('extra/bayesian_opt_params.json', 'w') as f:
    json.dump(all_results, f)

### Load Results and Explore

In [20]:
with open('extra/bayesian_opt_params.json', 'r') as f:
    all_results = json.load(f)

In [21]:
params = all_results['lgb']['max_params']
params['boosting'] = 'gbdt' if np.round(params['boosting'] == 0) else 'dart'
params['max_depth'] = int(np.round(params['max_depth']))
params['num_iterations'] = int(np.round(params['num_iterations']))
params['scale_pos_weight'] = int(np.round(params['scale_pos_weight']))
params

{'boosting': 'dart',
 'colsample_bytree': 0.21097363734776334,
 'learning_rate': 0.18490698058684293,
 'max_depth': 25,
 'num_iterations': 300,
 'scale_pos_weight': 5,
 'subsample': 0.010000000000000009}

In [31]:
cv = StratifiedKFold(4, True, SEED)
est = LGBMClassifier(**lgb_default_params, **params)
roc_auc = np.mean(cross_val_score(est, x_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1))
print(roc_auc)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.773688001487


### Explore and Add in Engineered Ftrs

In [22]:
eng_ftrs_train = pd.read_csv('clean_data/eng_ftrs_train.csv')
eng_ftrs_test = pd.read_csv('clean_data/eng_ftrs_test.csv')
eng_ftrs_train.shape, eng_ftrs_test.shape

((307511, 361), (48744, 361))

In [24]:
xtrain_all = pd.concat((train, eng_ftrs_train), axis=1)
xtest_all = pd.concat((test, eng_ftrs_test), axis=1)

xtrain_all.shape, xtest_all.shape

((307511, 886), (48744, 886))

In [65]:
# engineered features _alone_ do pretty well
cv = StratifiedKFold(N_CV, True, SEED)
est = LGBMClassifier(**lgb_default_params, **params)
roc_auc = np.mean(cross_val_score(est, eng_ftrs_train, y_train, scoring='roc_auc', cv=cv, n_jobs=-1))
print(roc_auc)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.748830500674


In [26]:
# :-/ combined no improvement
cv = StratifiedKFold(N_CV, True, SEED)
est = LGBMClassifier(**lgb_default_params, **params)
roc_auc = np.mean(cross_val_score(est, xtrain_all.values, y_train, scoring='roc_auc', cv=cv, n_jobs=-1))
print(roc_auc)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.770327243684


Filter to Use only Selected Features From Feature Selection

In [27]:
selected_ftrs = pd.read_csv('extra/all_selected_ftrs.csv', names=['ftr', 'score'])
selected_ftrs.head()

,ftr,score
0,ext_source_2,3.219939
1,ext_source_3,2.989627
2,amt_goods_price_application,2.297600
3,nb_probs,2.285854
4,name_income_type_Pensioner,2.122629


In [34]:
# don't need all 500 - same performance w 300
final_ftrs = selected_ftrs.ftr[:300]

In [33]:
# :-( still no improvement
cv = StratifiedKFold(N_CV, True, SEED)
est = LGBMClassifier(**lgb_default_params, **params)
roc_auc = np.mean(cross_val_score(est, xtrain_all[final_ftrs].values, y_train, scoring='roc_auc', cv=cv, n_jobs=-1))
print(roc_auc)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


0.770781418715


### Scratch Code for Generating Submission Files

In [81]:
params = all_results['lgb']['max_params']
params['boosting'] = 'gbdt' if np.round(params['boosting'] == 0) else 'dart'
params['max_depth'] = int(np.round(params['max_depth']))
params['n_iter'] = int(np.round(params['n_iter']))
params['scale_pos_weight'] = int(np.round(params['scale_pos_weight']))
params

{'boosting': 'dart',
 'colsample_bytree': 0.21097363734776334,
 'learning_rate': 0.18490698058684293,
 'max_depth': 25,
 'n_iter': 158,
 'scale_pos_weight': 5,
 'subsample': 0.010000000000000009}

In [71]:
lgb_clf = LGBMClassifier(**lgb_default_params, **params)
lgb_clf.fit(x_train, y_train)
probs = lgb_clf.predict_proba(x_test)

/home/ec2-user/anaconda3/lib/python3.6/site-packages/lightgbm/engine.py:102: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


In [72]:
submission = pd.DataFrame([test_id,probs[:,1]], index=['SK_ID_CURR', 'TARGET']).T

In [73]:
algo = 'lgbm'
submission.to_csv('submissions/{}_{}.csv'.format(algo, today.strftime('%Y%m%d')), index=False)

In [ ]:

# dtrain = xgb.DMatrix(x_train, label=y_train)
# nrounds = xgb_params.pop('nrounds', 75)
# clf = xgb.train(xgb_params, dtrain, nrounds)


In [ ]:
# clf.best_iteration

In [ ]:
# dtest = xgb.DMatrix(x_test)
# preds = clf.predict(dtest)

In [ ]:
# submission['TARGET'] = preds

In [ ]:
# algo = 'xgboost'
# submission.to_csv('submissions/{}_{}.csv'.format(algo, today.strftime('%Y%m%d')), index=False)